# Automatic differentiation with dual numbers

A dual number is defined as\
$
\begin{align} 
x = a + b \epsilon \tag{1}
\end{align}
$
where $\epsilon^2 = 0$. Here, the real part is $a$ and the dual part is $b$.\
Dual numbers are useful because they can be used in differentiation. Consider the square of a dual number:
\begin{align} 
x^2 = (a + b \epsilon)^2 = a^2 + 2ab\epsilon \tag{2}
\end{align}
Uusing the subsitution $a = x$ and $b = 1$, the dual part of $x^2$ is $2x$, which is the derivative of $x^2$ with respect to $x$.

## Example: differentiating a function.
Consider the function
$
\begin{align} 
f(x) = \log{(\sin{x})} + x^2 \cos{x} \tag{3}
\end{align}
$
The differentiated function is given by 
$
\begin{align} 
f'(x) = \cot{x} + 2x\cos{x} - x^2 \sin{x} \tag{4}
\end{align}
$
Let's compare the values of the differentiated function using both the analytical form of $f'(x)$ and by using dual numbers.

In [1]:
#import the required packages
import numpy as np
from dual_autodiff import Dual

In [2]:
x = Dual (2, {'x': 1})

In [3]:
#Define the above function and first order differentiated function such that it works on dual numbers
def f(x):
    return np.log(np.sin(x)) + x**2 * np.cos(x)

def f2(x):
    return 1 / np.tan(x) + 2 * x * np.cos(x) - x **2 * np.sin(x)

In [4]:
x = Dual (2, {'x': 1})
print(f(x))

Dual(real=-1.7596703822837303, dual={'x': np.float64(-5.759434607851582)})


## Example: adding custom functions

The tools.py file contains a tool_store that has all the base implementations of the class, but dual_autodiff allows the user to add custom functions. Let's demonstrate using the sigmoid function:
$
\begin{align} 
s(x) = \frac{1}{1+e^{-x}} \tag{5}
\end{align}
$

$
\begin{align} 
s'(x) = s(x) (1-s(x)) \tag{6}
\end{align}
$


In [7]:
from dual_autodiff.tools import add_function

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    s = sigmoid(x)
    return s * (1 - s)

# Add the function to the store
add_function('sigmoid', sigmoid, sigmoid_derivative)

# Now you can use it
x = Dual(2, {'x': 1})
result = sigmoid(x)
print(result)


TypeError: bad operand type for unary -: 'Dual'